In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

seed = 1337
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

In [3]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [4]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(max_words,)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [6]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [7]:
batch_size = 32
model.fit(x_train, y_train, batch_size=batch_size, epochs=5)
score = model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/5
8982/8982 [==============================] - 3s 375us/step - loss: 1.3942 - acc: 0.6902
Epoch 2/5
8982/8982 [==============================] - 3s 327us/step - loss: 0.7736 - acc: 0.8170
Epoch 3/5
8982/8982 [==============================] - 3s 325us/step - loss: 0.5559 - acc: 0.8662
Epoch 4/5
8982/8982 [==============================] - 3s 337us/step - loss: 0.4224 - acc: 0.8931
Epoch 5/5
2246/2246 [==============================] - 0s 55us/step


In [8]:
score[1]

0.80142475512021372

In [14]:
import h5py
model.save("model.h5")

In [17]:
import base64
!base64 model.h5 > model.h5.base64

'base64' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [18]:
import requests, json

def submit(submitterEmail,secret,key,submission_part, all_parts, data):
        submission = {
                    "assignmentKey": key,
                    "submitterEmail":  submitterEmail,
                    "secret":  secret,
                    "parts": {}
                  }
        for part in all_parts:
            if part == submission_part:
                submission["parts"][part] = {"output": data}
            else:
                submission["parts"][part] = dict()
        response = requests.post('https://www.coursera.org/api/onDemandProgrammingScriptSubmissions.v1', data=json.dumps(submission))
        if response.status_code == 201:
            print ("Submission successful, please check on the coursera grader page for the status")
        else:
            print ("Something went wrong, please have a look at the reponse of the grader")
            print ("-------------------------")
            print (response.text)
            print ("-------------------------")

In [20]:
key = "XbAMqtjdEeepUgo7OOVwng"
part = "LqPRQ"
email = "etienne.coyen@gmail.com"
secret = "Ofz0PSLcgRLOkoX8"

with open('model.h5.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
